## Question 1. dlt Version

In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [4]:
import importlib.metadata

print("Installed versions:")
print("qdrant-client:", importlib.metadata.version("qdrant-client"))
print("dlt:", importlib.metadata.version("dlt"))

Installed versions:
qdrant-client: 1.14.3
dlt: 1.13.0


In [5]:
import requests
import dlt
import qdrant_client

In [8]:
@dlt.resource(write_disposition="replace", name="zoomcamp_data")
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

### Question 2. dlt pipeline

In [6]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [9]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

c:\Users\mikes\anaconda3\envs\py11\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mikes\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

2025-07-10 22:09:38.002 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: [WinError 1314] A required privilege is not held by the client: '..\\..\\blobs\\a8b3208c2884c4efb86e49300fdd3dc877220cdf' -> 'C:\\Users\\mikes\\AppData\\Local\\Temp\\fastembed_cache\\models--Qdrant--bge-small-en\\snapshots\\8791246cc2a79c7949a4dc0d4a018cbd7d024879\\special_tokens_map.json' Falling back to other sources.
100%|██████████| 77.7M/77.7M [00:47<00:00, 1.62MiB/s]


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Run started at 2025-07-10 19:08:23.271399+00:00 and COMPLETED in 2 minutes and 17.12 seconds with 4 steps.
Step extract COMPLETED in 0.98 seconds.

Load package 1752174627.5965326 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.19 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752174627.5965326 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 11.61 seconds.
Pipeline zoomcamp_pipeline load step completed in 11.52 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used c:\Users\mikes\Documents\STUDY\LLM-zoomcamp\week-3\db.qdrant location to store data
Load package 1752174627.5965326 is LOADED and contains no failed jobs

Step run COMPLETED in 2 minutes and 17.12 seconds.
Pipeline zoomcamp_pipeline load step completed in 11.52 

### Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the meta.json file created in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.


In [10]:
with open("db.qdrant/meta.json", "r") as f:
    meta = f.read()
    model = meta.split('"vectors": {')[1].split(': {')[0].strip('"')
print(f"The model used for embeddings is: {model}")

The model used for embeddings is: fast-bge-small-en
